In [27]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

url = "http://www.cbr.ru/scripts/XML_daily.asp"

today = datetime.today()
today = today.strftime("%d/%m/%Y")

payload = {"date_req" : today}

responce = requests.get(url, params=payload)
#print(responce.text)
# print(responce.url)
xml = BeautifulSoup(responce.content, "lxml")

def get_val(ch):
	if ch == "RUR": return ("Российский рубль",1,1)
	if not ch in responce.text: return None
	a = responce.text.split(ch)[1].split("</Valute>")[0]
	nom = int(a.split("<Nominal>")[1].split("</Nominal>")[0])
	name = a.split("<Name>")[1].split("</Name>")[0]
	val = float(a.split("<Value>")[1].split("</Value>")[0].replace(",","."))
	return (name,nom,val)

def func(valute_from, valute_to,amount):
	valute_from = get_val(valute_from)
	valute_to = get_val(valute_to)
	if valute_to and valute_from:
		a = amount*valute_from[2]/valute_from[1]
		b = a*valute_to[1]/valute_to[2]
	return f"За {amount} {valute_from[0]} вы получите {b} {valute_to[0]}"
def getCourse (id):
	return xml.find("valute",  {'id': id}).value.text


print(getCourse("R01235"), "рублей за 1 доллар")
print(getCourse("R01239"), "рублей за 1 евро")
print(getCourse("R01375"), "рублей за 10 юаней")

print(func("EUR","USD",199))



74,7588 рублей за 1 доллар
79,6104 рублей за 1 евро
10,8409 рублей за 10 юаней
За 199 Евро вы получите 211.91444485465257 Доллар США


/Users/pattycha/Library/Python/3.8/lib/python/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
